In [1]:
import time
import torch
import numpy as np
import os

from scripts.model_builder import get_default_spec, save_model, load_model_only_inference
from scripts.transformer_prediction_interface import transformer_predict, get_params_from_config, TabPFNClassifier
from scripts.differentiable_pfn_evaluation import eval_model, eval_model_range

from datasets import load_openml_list, open_cc_dids, open_cc_valid_dids, test_dids_classification

from scripts import tabular_metrics
import random

from sklearn.model_selection import train_test_split

/anaconda/envs/tabpfn/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from sklearn.datasets import fetch_openml

In [3]:
data = fetch_openml(data_id=1111)

In [4]:
train_xs, test_xs, train_ys, test_ys = train_test_split(data.data, np.array(data.target), stratify=data.target, train_size=2000, test_size=2000)

In [5]:
train_xs.shape

(2000, 230)

In [6]:
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
ct = ColumnTransformer([("oe", make_pipeline(SimpleImputer(strategy='most_frequent'), OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)), make_column_selector(dtype_include="category"))], remainder="passthrough")

In [50]:
train_xs = ct.fit_transform(train_xs)
test_xs = ct.transform(test_xs)

In [51]:


classifier = TabPFNClassifier( model_string="num_features_1000_multiclass_02_18_2023_05_40_39", epoch=100,  N_ensemble_configurations=32)
print(classifier)
classifier.fit(train_xs, train_ys, overwrite_warning=True)
prediction_ = classifier.predict_proba(test_xs)
roc, ce = tabular_metrics.auc_metric(test_ys, prediction_), tabular_metrics.cross_entropy(test_ys, prediction_)
print('AUC', float(roc), 'Cross Entropy', float(ce))

loading model from file models_diff/prior_diff_real_checkpoint_num_features_1000_multiclass_02_18_2023_05_40_39_n_0_epoch_100.cpkt
Loading model that can be used for inference only
Using a Transformer with 26.29 M parameters
TabPFNClassifier(N_ensemble_configurations=32, epoch=100,
                 model_string='num_features_1000_multiclass_02_18_2023_05_40_39')


TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.

In [68]:
classifier = TabPFNClassifier(N_ensemble_configurations=32)
print(classifier)
classifier.fit(train_xs, train_ys, overwrite_warning=True)
prediction_ = classifier.predict_proba(test_xs)
roc, ce = tabular_metrics.auc_metric(test_ys, prediction_), tabular_metrics.cross_entropy(test_ys, prediction_)
print('AUC', float(roc), 'Cross Entropy', float(ce))

loading model from file models_diff/prior_diff_real_checkpoint_download_n_0_epoch_100.cpkt
Loading model that can be used for inference only
Using a Transformer with 25.82 M parameters
Multiple models in memory. This might lead to memory issues. Consider calling remove_models_from_memory()
TabPFNClassifier(N_ensemble_configurations=32)


ValueError: ('The number of features for this classifier is restricted to ', 100)

In [53]:
from sklearn.metrics import roc_auc_score
roc_auc_score(test_ys, prediction_[:, 1])

0.7967017424756733

In [64]:
si = SimpleImputer(add_indicator=True).fit(train_xs)

In [65]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs=-1).fit(si.transform(train_xs), train_ys)

In [66]:
roc_auc_score(test_ys, rf.predict_proba(si.transform(test_xs))[:, 1])

0.5334988119484046

In [7]:
data = fetch_openml("mnist_784")

In [8]:
train_xs, test_xs, train_ys, test_ys = train_test_split(np.array(data.data), np.array(data.target), stratify=data.target, train_size=2000, test_size=2000)

In [9]:
classifier = TabPFNClassifier(model_string="num_features_1000_multiclass_02_18_2023_05_40_39", epoch=100,  N_ensemble_configurations=32)
print(classifier)
classifier.fit(train_xs, train_ys, overwrite_warning=True)
classifier.score(test_xs, test_ys)

loading model from file models_diff/prior_diff_real_checkpoint_num_features_1000_multiclass_02_18_2023_05_40_39_n_0_epoch_100.cpkt
Loading model that can be used for inference only
Using a Transformer with 26.29 M parameters
TabPFNClassifier(N_ensemble_configurations=32, epoch=100,
                 model_string='num_features_1000_multiclass_02_18_2023_05_40_39')


0.9145

In [10]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs=-1).fit(train_xs, train_ys)
rf.score(test_xs, test_ys)

0.9125

In [11]:
from tabpfn.scripts.transformer_prediction_interface import NeighborsTabPFNClassifier, ApproxNNClassifier


In [17]:
ApproxNNClassifier().fit(train_xs, train_ys).score(test_xs, test_ys)

AttributeError: 'ApproxNNClassifier' object has no attribute 'classes_'

Bad pipe message: %s [b'6\xcc\xa0\xf8\xd2\t\x10#D\x9f\xd7\x81\t)P\x9a^\x9d \xe6\x00N\xfeb\xe06\xfd\xb6\xa1\xc6\xbe.\xfan\x99\xda\xc5lS\xf7\xfaQ}\xf2\xcd\x8a\xd9\xb5\x1b\x1b\x1c\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff']
Bad pipe message: %s [b'']
Bad pipe message: %s [b'\xd0\xb1\xdb\xe9\xcc\xd1\xa3v\xcd\x10\x1e\x01S\x1dD\t\xea\x04 \x96\x96q\xed\xff>\x17\xfaE\xd2a8\xef\x1eq%[\xd2\x899x\x1b\xa47\xdbGP\xa6\xbei\xf6\xbb\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17\x00\x00\x00\r\x00\x1e\x00\x1c\x04\x03\x05\x03\x06\x03\x08\x07\x08\x08\x08\t\x08\n\x08\x0b\x08\x04\x08\x05\x08\x06\x04\x01\x05\x01\x06\x01\x00+\x00\x03\x02\x03\x04\x00-\x00\x02\x01\x01\x003\x00&\x00$\x00\x1d\x00 \x04\x1eg\x0c\xefh[6\xc3\x97\x13\x17\xf8B\xb3\xb9']
Bad pipe message: %s [b'}\x1a\xab\xedH\xfc\xeb\xfe\xf8\x86\xa4\xfb\n.\x02G[\x8d\x00\x00\xa

In [15]:
NeighborsTabPFNClassifier(model_string="num_features_1000_multiclass_02_18_2023_05_40_39", epoch=100, predict_batch_size=100, device='cuda').fit(train_xs, train_ys).score(test_xs, test_ys)

loading model from file models_diff/prior_diff_real_checkpoint_num_features_1000_multiclass_02_18_2023_05_40_39_n_0_epoch_100.cpkt
Loading model that can be used for inference only
Using a Transformer with 26.29 M parameters


0.884

In [16]:
NeighborsTabPFNClassifier(model_string="num_features_1000_multiclass_02_18_2023_05_40_39", epoch=100, predict_batch_size=10, n_neighbors=100, device='cuda').fit(train_xs, train_ys).score(test_xs, test_ys)

loading model from file models_diff/prior_diff_real_checkpoint_num_features_1000_multiclass_02_18_2023_05_40_39_n_0_epoch_100.cpkt
Loading model that can be used for inference only
Using a Transformer with 26.29 M parameters


0.8965